In [1]:
import numpy as np
import pandas as pd 
pd.set_option('max_columns',1000)
pd.options.display.float_format = '{:,.4f}'.format
import matplotlib.pyplot as plt
import seaborn as sns
import gc
import time
import itertools
import os
import sys
import random
random.seed(42)
import warnings
warnings.filterwarnings('ignore')
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

In [2]:
# Memory saving function credit to https://www.kaggle.com/gemartin/load-data-reduce-memory-usage
def reduce_mem_usage(df):
    "iterate through all the columns of a dataframe and modify the data type to reduce memory usage."
    start_mem = df.memory_usage().sum() / 1024**2
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB --> {:.2f} MB (Decreased by {:.1f}%)'.format(
        start_mem, end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [3]:
Train = pd.read_csv('/Users/zhangyanpeng/Desktop/PUBG/pubg-finish-placement-prediction/train_V2.csv')
Train = reduce_mem_usage(Train)

Memory usage of dataframe is 983.90 MB --> 339.28 MB (Decreased by 65.5%)


In [4]:
train = Train.copy()
train.dropna(inplace=True)
train.shape

(4446965, 29)

In [18]:
train.sort_values('matchId')

,Id,groupId,matchId,assists,boosts,damageDealt,DBNOs,headshotKills,heals,killPlace,killPoints,kills,killStreaks,longestKill,matchDuration,matchType,maxPlace,numGroups,rankPoints,revives,rideDistance,roadKills,swimDistance,teamKills,vehicleDestroys,walkDistance,weaponsAcquired,winPoints,winPlacePerc
3599150,fc62a751955351,3a6addfa0df938,0000a43bce5eec,0,0,0.0000,0,0,0,95,1458,0,0,0.0000,1346,squad-fpp,28,28,0,0,0.0000,0,0.0000,0,0,9.6360,1,1557,0.0000
1185914,a638435c730f4e,6620b219ed2ee2,0000a43bce5eec,1,1,256.1000,3,0,0,38,1372,0,0,0.0000,1346,squad-fpp,28,28,0,1,"2,202.0000",0,0.0000,0,0,"2,400.0000",8,1559,0.7778
7931,a121348062f67a,7bd08592bb25e2,0000a43bce5eec,0,1,0.0000,0,0,1,41,1693,0,0,0.0000,1346,squad-fpp,28,28,0,0,0.0000,0,0.0000,0,0,"2,723.0000",3,1502,0.6667
918045,29ffb3ea02be3e,4bf06994bd4c9a,0000a43bce5eec,0,0,0.0000,0,0,0,60,1189,0,0,0.0000,1346,squad-fpp,28,28,0,0,0.0000,0,0.0000,0,0,347.7000,2,1433,0.3704
518453,c63253392e4fbb,b947dfc5cf2d02,0000a43bce5eec,0,0,0.0000,0,0,0,39,1163,0,0,0.0000,1346,squad-fpp,28,28,0,0,0.0000,0,0.0000,0,0,"1,402.0000",5,1521,0.7407
2841348,f806332c66c741,909fe185a8cdec,0000a43bce5eec,0,0,49.5000,0,0,0,72,1059,0,0,0.0000,1346,squad-fpp,28,28,0,0,0.0000,0,0.0000,0,0,123.3000,4,1460,0.1852
2357956,ffc5bf94121ebc,599d924f8a02db,0000a43bce5eec,0,3,19.3500,0,0,0,48,1000,0,0,0.0000,1346,squad-fpp,28,28,0,0,0.0000,0,50.8800,0,0,"2,438.0000",4,1500,0.5926
1136390,971fff07cfcb2a,dc8f83946f97a9,0000a43bce5eec,0,0,0.0000,0,0,0,77,1000,0,0,0.0000,1346,squad-fpp,28,28,0,0,0.0000,0,0.0000,0,0,23.9300,1,1500,0.1481
3832684,51bf1d40b394c6,6c44ef4381fe8d,0000a43bce5eec,0,1,289.7000,3,0,0,17,1224,2,1,80.0900,1346,squad-fpp,28,28,0,0,0.0000,0,0.0000,0,0,"1,807.0000",4,1423,0.7037
725551,d38d809a2f91d7,9cf70fed874fa1,0000a43bce5eec,0,3,54.1800,0,0,5,64,1573,0,0,0.0000,1346,squad-fpp,28,28,0,0,0.0000,0,0.0000,0,0,108.7000,2,1479,0.2963


In [21]:
def train_and_test_split(train):
    test = train[:1111742]
    train = train[1111742:]
    return train,test

In [22]:
train,test = train_and_test_split(train)
print(train.shape)
print(test.shape)

(3335223, 29)
(1111742, 29)


In [24]:
Test = test.copy()
print(Test.shape)
test.drop(['winPlacePerc'], axis=1, inplace=True)
print(test.shape)

(1111742, 29)
(1111742, 28)
